<a href="https://colab.research.google.com/github/larissa-eleterio/portfolio-larissa-eleterio/blob/master/Aplicando_an%C3%A1lise_de_componetes_principais_na_cria%C3%A7%C3%A3o_de_um_%C3%ADndice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Aplicando análise de componentes principais para criação de um índice

> Esse projeto de Data Science está relacionado ao notebook [*Analisando dados do Airbnb*](https://bit.ly/3d2iWrh) que publiquei anteriormente.
Nesse notebook irei elaborar de um índice que pudesse rankear as locações de acordo com algum critério multivariado. Para esse fim vamos utilizar a análise de componentes principais.



Proposta por Karl Pearsons a análise de componentes principais procura explicar a estrutura de variâncias-convariâncias de um grupo de $p$  variáveis a partir de $p$ combinações lineares das mesmas. Seus objetivos gerais são a redução de dimensão dos dados e a busca por interpretabilidade.

Apesar de serem necessárias $p$ componentes principais para reproduzir toda a variabilidade do conjunto, muita das vezes essa variabilidade pode ser bem representada por número menor de $k$ componentes principais. Desse modo teremos quase tanta informação a partir das $k$ componentes
principais quanto teríamos se utilizássemos as $p$ componentes principais originais.
A análise de componentes principais por vezes revela relações a princípio insuspeitas mas também interpretáveis que uma análise mais simplificada dos dados não mostraria.

Para realizar a análise de componentes principais tomamos um vetor aleatório $X = (X1;...;Xp)'$ com matriz de variâncias e covariâncias $\Sigma$. Extraímos então os autovalores $\lambda_1;...; \lambda_p$ e os autovetores $e_1,...,e_p$ desta mesma matriz $\Sigma$. Após isso, teremos então a matriz ortogonal $P=(e_1,...,e_p)$ e o vetor $Y = P'X$ será o vetor com as componentes principais de $\Sigma$.

Uma outra alternativa a utilizaçãoao da matriz de variâncias e covariâncias $\Sigma$ é a obtenção das componentes principais a partir da matriz de correlações $\rho$. Os passos para obtenção das componentes a partir da matriz $\rho$ são análogos. Em geral, as componentes principais obtidas a partir da matriz $\Sigma$ são diferntes das obtidas a partir da matriz $\rho$.

Vale ressaltar que quando as variáveis são medidas em escalas diferentes é mais natural estimar as componentes
principais a partir da matriz de correlações $\rho$.

Como esse projeto de Data Science é a continuação do notebook publicado anteriormente, não estarão descritos aqui os passos de tratamento e limpeza dos dados, porém você pode encontrar o passo a passo no meu [github](https://bit.ly/3d2iWrh)

Vamos fazer um recorte do banco selecionando apenas as variáveis quantitativas

In [0]:
df_pca = df_sem_outliers[['price', 'minimum_nights', 'number_of_reviews',
    'calculated_host_listings_count', 'availability_365']]


Um passo importante é a padronização dos dados, uma vez que os dados estão em 
escalas diferentes e se nao fosse aplicada a padronização seria necessário 
obter as componentes principais a partir da matriz de correlações e 
não a partir da matriz de variância e covariâncias

In [0]:
df_pca2 = scale(df_pca)

Nesse passo seleciona-se as duas primeiras componentes principais, as que agregam maior variabilidade

In [0]:
pca = PCA(n_components=2)

Finalmente a obtenção das componentes

In [0]:
principalComponents = pca.fit_transform(df_pca2)

Nessa etapa após as componentes terem sido extraídas
avaliação qual o percentual de variância explicada por cada componente principal. Isto é, o quanto da variabilidade dos dados pode ser explicado por cada componente.

É quase como uma medida de qualidade do ajuste.

In [18]:
eigenValues = pca.explained_variance_ratio_
eigenValues


array([0.25415911, 0.21611264])

Observe que a primeira componente principal possui 25,41% de variância explicada e a segunda componente principal 21,61% de variância explicada.

Vamos analisar agora as cargas fatoriais para entender por quais variáveis índice será bem definido.

In [0]:
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)

In [20]:
loading_matrix = pd.DataFrame(loadings, columns=['PC1','PC2'], index=df_pca.columns)
loading_matrix

,PC1,PC2
price,-0.732675,0.149572
minimum_nights,-0.577608,-0.121418
number_of_reviews,0.605323,-0.037898
calculated_host_listings_count,-0.068924,0.766604
availability_365,0.170952,0.674066


Observando a saíde acima podemos analiar a correlação entre a variável original e a componente principal. A variável *preço* e o primeiro componente principal PC1 têm uma correlação de $-0,73$, a variável *mínimo de noites* possui uma correlação de $-0,57$ e a variável *número de avaliações* uma correlação de $0,60$. Essas são as variáveis que melhor definem a primeira componente principal.

Na segunda componente principal, a variável *quantidade de imóveis disponíveis pelo anfitrião* e *disponibilidade ao longo do ano* possuem uma correlação de $0,76$ e $0,67$ com a segunda componente principal, respectivamente.

# Conclusão

É interessante perceber como a primeira componente principal está bem definida pelo preço e pelo mínimo de estadias com cargas fatoriais negativas e pelo número de avaliações de forma positiva.

O sinal das cargas fatorias não tem uma implicação qualitativa na interpretação, de forma que as acomodações com valores negativos nessa componente principal seriam aquelas mais caras e com maior quantidade mínima de noites.
As acomodações com valores positivos seriam aquelas com maior número de avaliações.

Logo, para escolher uma acomodação mais barata, que tenha menor quantidade de dias mínimos para estadias e com mais avaliações, deve-se priorizar os imóveis com valor positivo na primeira componente principal.

Essa componente principal também endossa as análises encontradas na seção de análise de correlações no notebook [Analisando dados do Airbnb](https://bit.ly/3d2iWrh) onde vimos que as variáveis número de avaliações e avaliações mensais são negativamente correlacionadas com a variável preço. Isso se deve ao fato de a técnica multivariada de análise de componentes principais estar relacionada a decomposição da matriz de variância e covariâncias.

A segunda componente principal está bem definida pela disponibilidade ao longo do ano e pelo número de imóveis que os afitriões são proprietários. Isso indica que acomodações com valores altos na segunda componente principal são aquelas com maior disponibilidade e cujos anfitriões colocam mais imóveis a disposição.

